# OpenSearch Korean Neural Sparse Model - Inference Test

학습된 한국어 Neural Sparse 모델을 로컬에서 테스트합니다.

## 테스트 항목
1. 모델 로드
2. 문서 인코딩 (BERT 기반)
3. 쿼리 인코딩 (IDF lookup - Inference-Free)
4. 유사도 계산 및 검색
5. 성능 벤치마크

## 1. 환경 설정 및 라이브러리 로드

In [1]:
import os
import json
import time
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
from torch import nn

# Transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForMaskedLM

# GPU/CPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

디바이스: cuda
GPU: Tesla T4
메모리: 14.57 GB


## 2. 모델 정의

In [2]:
class OpenSearchDocEncoder(nn.Module):
    """
    OpenSearch Neural Sparse Document Encoder (Doc-only mode)
    
    문서를 sparse vector로 인코딩하는 모델입니다.
    """
    def __init__(self, model_name="klue/bert-base"):
        super().__init__()
        
        # BERT 기반 인코더
        self.config = AutoConfig.from_pretrained(model_name)
        self.bert = AutoModelForMaskedLM.from_pretrained(model_name)
        
        self.vocab_size = self.config.vocab_size
        
        # Log saturation activation: log(1 + ReLU(x))
        self.activation = lambda x: torch.log1p(torch.relu(x))
    
    def forward(self, input_ids, attention_mask, return_dict=False):
        """
        Forward pass
        
        Args:
            input_ids: (batch_size, seq_len)
            attention_mask: (batch_size, seq_len)
        
        Returns:
            sparse_vector: (batch_size, vocab_size)
        """
        # BERT MLM head output
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        
        # Logits: (batch_size, seq_len, vocab_size)
        logits = outputs.logits
        
        # Apply activation: log(1 + ReLU(logits))
        activated = self.activation(logits)
        
        # Max pooling over sequence length
        # (batch_size, seq_len, vocab_size) → (batch_size, vocab_size)
        sparse_vector = torch.max(
            activated * attention_mask.unsqueeze(-1),
            dim=1
        ).values
        
        if return_dict:
            return {'output': sparse_vector}
        
        return sparse_vector

print("✓ 모델 클래스 정의 완료")

✓ 모델 클래스 정의 완료


## 3. 저장된 모델 로드

In [3]:
# 모델 디렉토리 경로
MODEL_DIR = "./opensearch-korean-neural-sparse-v1"

# 파일 존재 확인
required_files = [
    "pytorch_model.bin",
    "idf.json",
    "config.json",
    "tokenizer.json",
    "vocab.txt"
]

print("📦 모델 파일 확인...")
for filename in required_files:
    filepath = os.path.join(MODEL_DIR, filename)
    if os.path.exists(filepath):
        size = os.path.getsize(filepath) / 1024
        print(f"  ✓ {filename:25s} ({size:>10.2f} KB)")
    else:
        print(f"  ✗ {filename:25s} [NOT FOUND]")
        raise FileNotFoundError(f"Required file not found: {filename}")

print("\n" + "="*60)
print("모델 로딩 중...")
print("="*60)

# 1. Config 로드
with open(os.path.join(MODEL_DIR, "config.json"), 'r', encoding='utf-8') as f:
    model_config = json.load(f)

print("\n✓ Config 로드 완료")
print(f"  모델 타입: {model_config['model_type']}")
print(f"  베이스 모델: {model_config['base_model']}")
print(f"  Vocab size: {model_config['vocab_size']:,}")
print(f"  Max length: {model_config['max_seq_length']}")

# 2. Tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
print(f"\n✓ Tokenizer 로드 완료")
print(f"  Vocab size: {tokenizer.vocab_size:,}")

# 3. IDF 딕셔너리 로드
with open(os.path.join(MODEL_DIR, "idf.json"), 'r', encoding='utf-8') as f:
    idf_dict = json.load(f)

print(f"\n✓ IDF 딕셔너리 로드 완료")
print(f"  토큰 수: {len(idf_dict):,}")
print(f"  평균 IDF: {np.mean(list(idf_dict.values())):.4f}")

# 4. Document Encoder 모델 로드
base_model = model_config['base_model']
doc_encoder = OpenSearchDocEncoder(base_model)

# 학습된 가중치 로드
state_dict = torch.load(
    os.path.join(MODEL_DIR, "pytorch_model.bin"),
    map_location=device
)
doc_encoder.load_state_dict(state_dict)
doc_encoder = doc_encoder.to(device)
doc_encoder.eval()  # Evaluation mode

print(f"\n✓ Document Encoder 로드 완료")
print(f"  Parameters: {sum(p.numel() for p in doc_encoder.parameters()):,}")
print(f"  Device: {device}")

print("\n" + "="*60)
print("✅ 모델 로드 완료!")
print("="*60)

📦 모델 파일 확인...
  ✓ pytorch_model.bin         ( 432313.86 KB)
  ✓ idf.json                  (    905.19 KB)
  ✓ config.json               (      0.55 KB)
  ✓ tokenizer.json            (    734.63 KB)
  ✓ vocab.txt                 (    242.65 KB)

모델 로딩 중...

✓ Config 로드 완료
  모델 타입: opensearch-neural-sparse-doc-encoder
  베이스 모델: klue/bert-base
  Vocab size: 32,000
  Max length: 128

✓ Tokenizer 로드 완료
  Vocab size: 32,000

✓ IDF 딕셔너리 로드 완료
  토큰 수: 29,205
  평균 IDF: 8.9035


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.


✓ Document Encoder 로드 완료
  Parameters: 110,650,880
  Device: cuda

✅ 모델 로드 완료!


## 4. Inference 함수 정의

In [4]:
def encode_document(text, model, tokenizer, device, max_length=128):
    """
    문서를 sparse vector로 인코딩 (모델 사용 - 느림)
    
    Args:
        text: 문서 텍스트
        model: Document encoder 모델
        tokenizer: Tokenizer
        device: 디바이스
        max_length: 최대 시퀀스 길이
    
    Returns:
        sparse_vector: (vocab_size,) numpy array
    """
    model.eval()
    
    # 토큰화
    encoded = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    
    # Inference
    with torch.no_grad():
        sparse_vec = model(input_ids, attention_mask)
    
    return sparse_vec.cpu().numpy()[0]


def encode_query_inference_free(text, tokenizer, idf_dict, max_length=128):
    """
    쿼리를 sparse vector로 인코딩 (IDF lookup - Inference-Free! 매우 빠름)
    
    Args:
        text: 쿼리 텍스트
        tokenizer: Tokenizer
        idf_dict: IDF 딕셔너리
        max_length: 최대 시퀀스 길이
    
    Returns:
        sparse_vector: (vocab_size,) numpy array
    """
    # 토큰화
    tokens = tokenizer.encode(
        text,
        add_special_tokens=False,
        max_length=max_length,
        truncation=True
    )
    
    # IDF lookup
    sparse_vec = np.zeros(tokenizer.vocab_size)
    for token_id in tokens:
        token_str = tokenizer.decode([token_id])
        if token_str in idf_dict:
            sparse_vec[token_id] = idf_dict[token_str]
    
    return sparse_vec


def get_top_tokens(sparse_vec, tokenizer, top_k=15):
    """
    Sparse vector에서 상위 토큰 추출
    
    Args:
        sparse_vec: (vocab_size,) numpy array
        tokenizer: Tokenizer
        top_k: 상위 k개
    
    Returns:
        List of (token, value) tuples
    """
    top_indices = np.argsort(sparse_vec)[-top_k:][::-1]
    top_values = sparse_vec[top_indices]
    
    top_tokens = []
    for idx, val in zip(top_indices, top_values):
        if val > 0:
            token = tokenizer.decode([idx])
            top_tokens.append((token, val))
    
    return top_tokens


def compute_similarity(query_vec, doc_vec):
    """
    쿼리와 문서의 유사도 계산 (Dot product)
    
    Args:
        query_vec: (vocab_size,) numpy array
        doc_vec: (vocab_size,) numpy array
    
    Returns:
        similarity score (float)
    """
    return np.dot(query_vec, doc_vec)


print("✓ Inference 함수 정의 완료")

✓ Inference 함수 정의 완료


## 5. 문서 인코딩 테스트

In [5]:
# 테스트 문서
test_documents = [
    "OpenSearch는 강력한 오픈소스 검색 및 분석 엔진입니다. 벡터 검색과 neural sparse 검색을 지원합니다.",
    "한국어 자연어 처리는 형태소 분석, 품사 태깅, 개체명 인식 등을 포함합니다. KoNLPy와 같은 라이브러리가 널리 사용됩니다.",
    "딥러닝 모델 학습을 위해서는 충분한 데이터와 컴퓨팅 자원이 필요합니다. GPU를 사용하면 학습 속도가 크게 향상됩니다.",
    "ChatGPT는 OpenAI가 개발한 대화형 AI 모델입니다. GPT 아키텍처를 기반으로 하며 다양한 작업을 수행할 수 있습니다.",
    "벡터 검색은 임베딩 기반으로 의미적 유사도를 계산하여 검색합니다. Dense retrieval 방식이라고도 불립니다.",
]

print("="*60)
print("📄 문서 인코딩 테스트 (Model Inference)")
print("="*60)

doc_vectors = []
encoding_times = []

for i, doc in enumerate(test_documents, 1):
    print(f"\n[문서 {i}]")
    print(f"텍스트: {doc[:60]}...")
    
    # 인코딩 시간 측정
    start_time = time.time()
    sparse_vec = encode_document(doc, doc_encoder, tokenizer, device)
    elapsed = time.time() - start_time
    encoding_times.append(elapsed)
    
    doc_vectors.append(sparse_vec)
    
    # 통계
    non_zero = np.count_nonzero(sparse_vec)
    sparsity = (1 - non_zero / len(sparse_vec)) * 100
    l1_norm = np.sum(np.abs(sparse_vec))
    
    print(f"  희소성: {sparsity:.2f}% (non-zero: {non_zero:,}/{len(sparse_vec):,})")
    print(f"  L1 Norm: {l1_norm:.2f}")
    print(f"  인코딩 시간: {elapsed*1000:.2f}ms")
    
    # 상위 토큰
    print(f"  상위 토큰:")
    top_tokens = get_top_tokens(sparse_vec, tokenizer, top_k=10)
    for j, (token, value) in enumerate(top_tokens, 1):
        print(f"    {j:2d}. {token:15s} ({value:.4f})")

print(f"\n평균 인코딩 시간: {np.mean(encoding_times)*1000:.2f}ms")

📄 문서 인코딩 테스트 (Model Inference)

[문서 1]
텍스트: OpenSearch는 강력한 오픈소스 검색 및 분석 엔진입니다. 벡터 검색과 neural sparse 검색을...
  희소성: 99.98% (non-zero: 7/32,000)
  L1 Norm: 1.82
  인코딩 시간: 142.29ms
  상위 토큰:
     1. ##earch         (0.4343)
     2. 검색              (0.3980)
     3. 분석              (0.3320)
     4. 보고서             (0.2745)
     5. ##S             (0.2496)
     6. 탐색              (0.1086)
     7. 오               (0.0222)

[문서 2]
텍스트: 한국어 자연어 처리는 형태소 분석, 품사 태깅, 개체명 인식 등을 포함합니다. KoNLPy와 같은 라이브러리...
  희소성: 99.98% (non-zero: 5/32,000)
  L1 Norm: 1.93
  인코딩 시간: 12.97ms
  상위 토큰:
     1. 언어              (0.7562)
     2. 처리              (0.7488)
     3. 자연              (0.3232)
     4. 라이브             (0.0915)
     5. ##러리            (0.0058)

[문서 3]
텍스트: 딥러닝 모델 학습을 위해서는 충분한 데이터와 컴퓨팅 자원이 필요합니다. GPU를 사용하면 학습 속도가 크게 ...
  희소성: 99.97% (non-zero: 9/32,000)
  L1 Norm: 5.01
  인코딩 시간: 12.33ms
  상위 토큰:
     1. 딥               (1.1729)
     2. 학습              (0.9816)
     3. GP              (0.8123)
     4. 

## 6. 쿼리 인코딩 테스트 (Inference-Free)

In [6]:
# 테스트 쿼리
test_queries = [
    "OpenSearch 벡터 검색",
    "한국어 자연어 처리 기술",
    "딥러닝 GPU 학습",
    "ChatGPT LLM 모델",
    "검색 엔진 최적화",
    "인공지능 프롬프트 엔지니어링",
]

print("="*60)
print("📝 쿼리 인코딩 테스트 (Inference-Free: Tokenizer + IDF Lookup)")
print("="*60)

query_vectors = []
query_encoding_times = []

for i, query in enumerate(test_queries, 1):
    print(f"\n[쿼리 {i}] {query}")
    
    # 인코딩 시간 측정
    start_time = time.time()
    sparse_vec = encode_query_inference_free(query, tokenizer, idf_dict)
    elapsed = time.time() - start_time
    query_encoding_times.append(elapsed)
    
    query_vectors.append(sparse_vec)
    
    # 통계
    non_zero = np.count_nonzero(sparse_vec)
    sparsity = (1 - non_zero / len(sparse_vec)) * 100
    
    print(f"  희소성: {sparsity:.2f}% (non-zero: {non_zero}/{len(sparse_vec):,})")
    print(f"  인코딩 시간: {elapsed*1000:.4f}ms ⚡ (매우 빠름!)")
    
    # 상위 토큰
    print(f"  상위 토큰:")
    top_tokens = get_top_tokens(sparse_vec, tokenizer, top_k=10)
    for j, (token, value) in enumerate(top_tokens, 1):
        print(f"    {j:2d}. {token:15s} ({value:.4f})")

print(f"\n평균 쿼리 인코딩 시간: {np.mean(query_encoding_times)*1000:.4f}ms")
print(f"\n💡 문서 인코딩 대비 속도: {np.mean(encoding_times)/np.mean(query_encoding_times):.1f}배 빠름!")

📝 쿼리 인코딩 테스트 (Inference-Free: Tokenizer + IDF Lookup)

[쿼리 1] OpenSearch 벡터 검색
  희소성: 99.98% (non-zero: 6/32,000)
  인코딩 시간: 0.5574ms ⚡ (매우 빠름!)
  상위 토큰:
     1. 벡               (12.2485)
     2. 검색              (8.8385)
     3. Op              (7.4678)
     4. ##S             (6.8928)
     5. ##터             (6.4828)
     6. ##en            (5.7666)

[쿼리 2] 한국어 자연어 처리 기술
  희소성: 99.98% (non-zero: 5/32,000)
  인코딩 시간: 0.2050ms ⚡ (매우 빠름!)
  상위 토큰:
     1. 한국어             (7.6927)
     2. 자연              (6.2214)
     3. 처리              (6.0865)
     4. 기술              (5.1364)
     5. ##어             (3.3754)

[쿼리 3] 딥러닝 GPU 학습
  희소성: 99.98% (non-zero: 6/32,000)
  인코딩 시간: 0.2315ms ⚡ (매우 빠름!)
  상위 토큰:
     1. GP              (13.5708)
     2. ##닝             (12.3340)
     3. 딥               (10.4745)
     4. 학습              (7.6003)
     5. ##러             (6.9786)
     6. ##U             (5.9254)

[쿼리 4] ChatGPT LLM 모델
  희소성: 99.97% (non-zero: 9/32,000)
  인코딩 시간: 0.2255ms ⚡ (매우 빠름!)
  상위 

## 7. 검색 테스트 (유사도 계산)

In [7]:
print("="*60)
print("🔍 검색 테스트 (Query-Document Similarity)")
print("="*60)

# 각 쿼리에 대해 가장 유사한 문서 찾기
for i, query in enumerate(test_queries):
    print(f"\n{'='*60}")
    print(f"쿼리: {query}")
    print(f"{'='*60}")
    
    query_vec = query_vectors[i]
    
    # 모든 문서와의 유사도 계산
    similarities = []
    for j, doc_vec in enumerate(doc_vectors):
        sim = compute_similarity(query_vec, doc_vec)
        similarities.append((j, sim))
    
    # 유사도 순으로 정렬
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # 상위 3개 문서 출력
    print("\n검색 결과 (상위 3개):")
    for rank, (doc_idx, score) in enumerate(similarities[:3], 1):
        print(f"\n  [{rank}위] 유사도: {score:.4f}")
        print(f"       문서: {test_documents[doc_idx][:80]}...")

🔍 검색 테스트 (Query-Document Similarity)

쿼리: OpenSearch 벡터 검색

검색 결과 (상위 3개):

  [1위] 유사도: 19.4390
       문서: 벡터 검색은 임베딩 기반으로 의미적 유사도를 계산하여 검색합니다. Dense retrieval 방식이라고도 불립니다....

  [2위] 유사도: 5.2380
       문서: OpenSearch는 강력한 오픈소스 검색 및 분석 엔진입니다. 벡터 검색과 neural sparse 검색을 지원합니다....

  [3위] 유사도: 0.0000
       문서: 한국어 자연어 처리는 형태소 분석, 품사 태깅, 개체명 인식 등을 포함합니다. KoNLPy와 같은 라이브러리가 널리 사용됩니다....

쿼리: 한국어 자연어 처리 기술

검색 결과 (상위 3개):

  [1위] 유사도: 6.5684
       문서: 한국어 자연어 처리는 형태소 분석, 품사 태깅, 개체명 인식 등을 포함합니다. KoNLPy와 같은 라이브러리가 널리 사용됩니다....

  [2위] 유사도: 0.0000
       문서: OpenSearch는 강력한 오픈소스 검색 및 분석 엔진입니다. 벡터 검색과 neural sparse 검색을 지원합니다....

  [3위] 유사도: 0.0000
       문서: 딥러닝 모델 학습을 위해서는 충분한 데이터와 컴퓨팅 자원이 필요합니다. GPU를 사용하면 학습 속도가 크게 향상됩니다....

쿼리: 딥러닝 GPU 학습

검색 결과 (상위 3개):

  [1위] 유사도: 35.5924
       문서: 딥러닝 모델 학습을 위해서는 충분한 데이터와 컴퓨팅 자원이 필요합니다. GPU를 사용하면 학습 속도가 크게 향상됩니다....

  [2위] 유사도: 0.0000
       문서: OpenSearch는 강력한 오픈소스 검색 및 분석 엔진입니다. 벡터 검색과 neural sparse 검색을 지원합니다....

  [3위] 유사도: 0.0000
       문서: 한국어 자

## 8. 성능 벤치마크

In [8]:
print("="*60)
print("⚡ 성능 벤치마크")
print("="*60)

# 대량 쿼리 처리 속도 테스트
num_queries = 100
test_query_text = "OpenSearch 벡터 검색 최적화"

print(f"\n테스트: {num_queries}개 쿼리 인코딩 (Inference-Free)")
start_time = time.time()
for _ in range(num_queries):
    _ = encode_query_inference_free(test_query_text, tokenizer, idf_dict)
elapsed = time.time() - start_time

print(f"총 시간: {elapsed:.4f}s")
print(f"평균 시간: {elapsed/num_queries*1000:.4f}ms per query")
print(f"처리량: {num_queries/elapsed:.2f} queries/sec")

# 문서 인코딩 속도 테스트
num_docs = 10
test_doc_text = "OpenSearch는 강력한 검색 엔진입니다."

print(f"\n테스트: {num_docs}개 문서 인코딩 (Model Inference)")
start_time = time.time()
for _ in range(num_docs):
    _ = encode_document(test_doc_text, doc_encoder, tokenizer, device)
elapsed = time.time() - start_time

print(f"총 시간: {elapsed:.4f}s")
print(f"평균 시간: {elapsed/num_docs*1000:.2f}ms per document")
print(f"처리량: {num_docs/elapsed:.2f} documents/sec")

print("\n" + "="*60)
print("💡 요약")
print("="*60)
print("✓ 쿼리 인코딩: 매우 빠름 (IDF lookup만 사용)")
print("✓ 문서 인코딩: 느림 (BERT 모델 사용)")
print("✓ 쿼리는 실시간 처리 가능, 문서는 사전 인덱싱 필요")

⚡ 성능 벤치마크

테스트: 100개 쿼리 인코딩 (Inference-Free)
총 시간: 0.0532s
평균 시간: 0.5324ms per query
처리량: 1878.35 queries/sec

테스트: 10개 문서 인코딩 (Model Inference)
총 시간: 0.1374s
평균 시간: 13.74ms per document
처리량: 72.76 documents/sec

💡 요약
✓ 쿼리 인코딩: 매우 빠름 (IDF lookup만 사용)
✓ 문서 인코딩: 느림 (BERT 모델 사용)
✓ 쿼리는 실시간 처리 가능, 문서는 사전 인덱싱 필요


## 9. 커스텀 테스트

In [9]:
# 사용자 정의 쿼리와 문서로 테스트
custom_query = "인공지능 LLM 모델 최적화 방법"  # 여기에 테스트할 쿼리 입력

custom_documents = [
    "LLM 모델 최적화를 위해서는 양자화, 프루닝, 지식 증류 등의 기법을 사용할 수 있습니다.",
    "한국어 음식 중에서 김치찌개는 가장 인기 있는 메뉴 중 하나입니다.",
    "대형 언어 모델의 추론 속도를 높이기 위해 KV 캐시와 배치 처리를 활용합니다.",
]  # 여기에 테스트할 문서 입력

print("="*60)
print("🧪 커스텀 검색 테스트")
print("="*60)
print(f"\n쿼리: {custom_query}")

# 쿼리 인코딩
query_vec = encode_query_inference_free(custom_query, tokenizer, idf_dict)

# 문서 인코딩 및 유사도 계산
results = []
for i, doc in enumerate(custom_documents):
    doc_vec = encode_document(doc, doc_encoder, tokenizer, device)
    similarity = compute_similarity(query_vec, doc_vec)
    results.append((i, similarity, doc))

# 유사도 순으로 정렬
results.sort(key=lambda x: x[1], reverse=True)

print("\n검색 결과:")
for rank, (doc_idx, score, doc_text) in enumerate(results, 1):
    print(f"\n[{rank}위] 유사도: {score:.4f}")
    print(f"     문서: {doc_text}")

🧪 커스텀 검색 테스트

쿼리: 인공지능 LLM 모델 최적화 방법

검색 결과:

[1위] 유사도: 12.0368
     문서: LLM 모델 최적화를 위해서는 양자화, 프루닝, 지식 증류 등의 기법을 사용할 수 있습니다.

[2위] 유사도: 2.3374
     문서: 대형 언어 모델의 추론 속도를 높이기 위해 KV 캐시와 배치 처리를 활용합니다.

[3위] 유사도: 0.0000
     문서: 한국어 음식 중에서 김치찌개는 가장 인기 있는 메뉴 중 하나입니다.


## 10. 희소성(Sparsity) 분석

In [10]:
print("="*60)
print("📊 희소성 분석")
print("="*60)

# 문서 벡터 희소성
doc_sparsities = []
for vec in doc_vectors:
    non_zero = np.count_nonzero(vec)
    sparsity = (1 - non_zero / len(vec)) * 100
    doc_sparsities.append(sparsity)

print(f"\n문서 벡터 희소성:")
print(f"  평균: {np.mean(doc_sparsities):.2f}%")
print(f"  최소: {np.min(doc_sparsities):.2f}%")
print(f"  최대: {np.max(doc_sparsities):.2f}%")
print(f"  표준편차: {np.std(doc_sparsities):.2f}%")

# 쿼리 벡터 희소성
query_sparsities = []
for vec in query_vectors:
    non_zero = np.count_nonzero(vec)
    sparsity = (1 - non_zero / len(vec)) * 100
    query_sparsities.append(sparsity)

print(f"\n쿼리 벡터 희소성:")
print(f"  평균: {np.mean(query_sparsities):.2f}%")
print(f"  최소: {np.min(query_sparsities):.2f}%")
print(f"  최대: {np.max(query_sparsities):.2f}%")
print(f"  표준편차: {np.std(query_sparsities):.2f}%")

print(f"\n💡 희소성이 높을수록 (100%에 가까울수록):")
print(f"   - 메모리 효율적")
print(f"   - 검색 속도 빠름")
print(f"   - 저장 공간 적음")

📊 희소성 분석

문서 벡터 희소성:
  평균: 99.98%
  최소: 99.97%
  최대: 99.99%
  표준편차: 0.01%

쿼리 벡터 희소성:
  평균: 99.98%
  최소: 99.97%
  최대: 99.99%
  표준편차: 0.01%

💡 희소성이 높을수록 (100%에 가까울수록):
   - 메모리 효율적
   - 검색 속도 빠름
   - 저장 공간 적음


## 11. 결과 요약

In [11]:
print("="*60)
print("✅ OpenSearch Korean Neural Sparse Model - Inference Test 완료")
print("="*60)

print("\n📊 테스트 요약:")
print(f"  • 문서 인코딩 평균 시간: {np.mean(encoding_times)*1000:.2f}ms")
print(f"  • 쿼리 인코딩 평균 시간: {np.mean(query_encoding_times)*1000:.4f}ms")
print(f"  • 속도 차이: {np.mean(encoding_times)/np.mean(query_encoding_times):.1f}배")
print(f"  • 문서 벡터 평균 희소성: {np.mean(doc_sparsities):.2f}%")
print(f"  • 쿼리 벡터 평균 희소성: {np.mean(query_sparsities):.2f}%")

print("\n🎯 핵심 특징:")
print("  ✓ Inference-Free 쿼리 인코딩 (매우 빠름)")
print("  ✓ 높은 희소성 (메모리 효율적)")
print("  ✓ 의미 기반 검색 가능")
print("  ✓ 한국어 최적화")

print("\n💡 OpenSearch 통합:")
print("  1. 문서는 사전에 인코딩하여 인덱싱")
print("  2. 쿼리는 실시간으로 IDF lookup")
print("  3. rank_features 필드로 저장")
print("  4. neural_sparse 쿼리로 검색")

print("\n" + "="*60)

✅ OpenSearch Korean Neural Sparse Model - Inference Test 완료

📊 테스트 요약:
  • 문서 인코딩 평균 시간: 38.40ms
  • 쿼리 인코딩 평균 시간: 0.2569ms
  • 속도 차이: 149.4배
  • 문서 벡터 평균 희소성: 99.98%
  • 쿼리 벡터 평균 희소성: 99.98%

🎯 핵심 특징:
  ✓ Inference-Free 쿼리 인코딩 (매우 빠름)
  ✓ 높은 희소성 (메모리 효율적)
  ✓ 의미 기반 검색 가능
  ✓ 한국어 최적화

💡 OpenSearch 통합:
  1. 문서는 사전에 인코딩하여 인덱싱
  2. 쿼리는 실시간으로 IDF lookup
  3. rank_features 필드로 저장
  4. neural_sparse 쿼리로 검색

